## READ FILE


In [1]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd
import numpy as np

# Define file path and sheet names
file_path = "ULSP-instancesR.xlsx"
sheet_list = ["6-periods (1)", "6-periods (2)", "12-periods (1)", "12-periods (2)", 
              "24-periods (1)", "24-periods (2)", "52-periods (1)", "52-periods (2)", 
              "104-periods (1)", "104-periods (2)"]

# DataFrame to store all results
results_df = pd.DataFrame(columns=["Sheet", "Period", "y", "x", "S", "b"])

# Process each sheet
for sheet_name in sheet_list:
    print(f"\nProcessing sheet: {sheet_name}")

    # Read the Excel file from the specific sheet
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Insert dummy period 0
    demand_forecast = np.insert(df["Demand Forecast"].to_numpy(), 0, 0)
    setup_cost = np.insert(df["Setup Cost"].to_numpy(), 0, 0)
    production_cost = np.insert(df["Production cost"].to_numpy(), 0, 0)
    holding_cost = np.insert(df["Holding cost"].to_numpy(), 0, 0)
    backlogging_cost = np.insert(df["Backlogging cost"].to_numpy(), 0, 0)

    T = df.shape[0]
    print(f"Number of periods: {T}")

    # Create Gurobi model
    model = gp.Model("Uncapacitated Lot-Sizing Problem")

    # Decision variables
    y = model.addVars(T+1, vtype=GRB.BINARY, name="y")  
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="x")  
    S = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="S")  
    b = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="b")  # Amount backlogged at the end of period t

    # Objective function
    model.setObjective(
            quicksum(setup_cost[t] * y[t] for t in range(1, T+1)) +
            quicksum(production_cost[t] * x[t] for t in range(1, T+1)) +
            quicksum(holding_cost[t] * S[t] for t in range(1, T+1)) +
            quicksum(backlogging_cost[t] * b[t] for t in range(1, T+1)),
            GRB.MINIMIZE
        )

    # Constraints
    model.addConstr(S[0] == 0, name="no_inventory0")
    model.addConstr(S[T] == 0, name="no_inventoryT")
    model.addConstr(b[0] == 0, name="no_backlogging0")
    model.addConstr(b[T] == 0, name="no_backloggingT")

    for t in range(1, T+1):
        model.addConstr(x[t] + S[t-1] - b[t-1] == demand_forecast[t] + S[t] - b[t], name=f"demand_satisfied_{t}")
        model.addConstr(x[t] <= (quicksum(demand_forecast[m] for m in range(t, T+1)) + b[t]) * y[t], name=f"setup_constraint_{t}")

    # Solve model
    model.optimize()

    # Check if the model found an optimal solution
    if model.status == GRB.OPTIMAL:
        print("\nOptimal solution found:")
        # Create a temporary DataFrame for the current sheet results
        sheet_results = pd.DataFrame(columns=["Sheet", "Period", "y", "x", "S", "b"])
        for t in range(1, T+1):
            # Store the results for the current sheet
            sheet_results = pd.concat([sheet_results, pd.DataFrame([{
                "Sheet": sheet_name,
                "Period": t,
                "y": y[t].X,
                "x": x[t].X,
                "S": S[t].X,
                "b": b[t].X
            }])], ignore_index=True)
        
        # Concatenate the results for all sheets
        results_df = pd.concat([results_df, sheet_results], ignore_index=True)

    else:
        print(f"No optimal solution found for sheet {sheet_name}.")

# Save results to a CSV file
results_df.to_csv("optimization_results.csv", index=False)
print("\nResults have been saved to 'optimization_results.csv'.")


Processing sheet: 6-periods (1)
Number of periods: 6
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2560044
Academic license 2560044 - for non-commercial use only - registered to vi___@ugent.be
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2560044 - for non-commercial use only - registered to vi___@ugent.be
Optimize a model with 10 rows, 28 columns and 34 nonzeros
Model fingerprint: 0xf02f4325
Model has 6 quadratic constraints
Variable types: 21 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+03]
  Objective range  [3e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 6e+02]
Presolve removed 4 rows and 7 column

C:\Users\irako\AppData\Local\Temp\ipykernel_18780\978396970.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{
C:\Users\irako\AppData\Local\Temp\ipykernel_18780\978396970.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, sheet_results], ignore_index=True)



Explored 1 nodes (16 simplex iterations) in 0.06 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 3: 105079 107416 107416 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.050790000000e+05, best bound 1.050790000000e+05, gap 0.0000%

Optimal solution found:

Processing sheet: 12-periods (1)
Number of periods: 12
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i5-8250U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2560044 - for non-commercial use only - registered to vi___@ugent.be
Optimize a model with 16 rows, 52 columns and 64 nonzeros
Model fingerprint: 0xbee7b666
Model has 12 quadratic constraints
Variable types: 39 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 5e+03

C:\Users\irako\AppData\Local\Temp\ipykernel_18780\978396970.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{


     0     0 40067.1352    0   17          - 40067.1352      -     -    0s
     0     0 46366.3426    0   19          - 46366.3426      -     -    0s
H    0     0                    148569.00000 46959.9097  68.4%     -    0s
H    0     0                    131615.00000 46959.9097  64.3%     -    0s
H    0     0                    90055.652174 46959.9097  47.9%     -    0s
H    0     0                    89639.000000 46959.9097  47.6%     -    0s
H    0     0                    70618.000000 46959.9097  33.5%     -    0s
     0     0 46959.9097    0   19 70618.0000 46959.9097  33.5%     -    0s
H    0     0                    68124.000000 46959.9097  31.1%     -    0s
     0     0 48209.9559    0   20 68124.0000 48209.9559  29.2%     -    0s
H    0     0                    64381.000000 48209.9559  25.1%     -    0s
H    0     0                    55961.000000 48209.9559  13.9%     -    0s
     0     0 50203.0720    0   20 55961.0000 50203.0720  10.3%     -    0s
     0     0 50203.0720  

...

C:\Users\irako\AppData\Local\Temp\ipykernel_18780\978396970.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{


Presolve removed 5 rows and 9 columns
Presolve time: 0.00s
Presolved: 412 rows, 720 columns, 1335 nonzeros
Presolved model has 206 SOS constraint(s)
Variable types: 514 continuous, 206 integer (206 binary)

Root relaxation: objective 2.234785e+05, 433 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 223478.521    0  164          - 223478.521      -     -    0s
H    0     0                    1.096822e+07 223478.521  98.0%     -    0s
H    0     0                    6203560.9998 223478.521  96.4%     -    0s
H    0     0                    4332525.0000 223478.521  94.8%     -    0s
     0     0 288412.522    0  138 4332525.00 288412.522  93.3%     -    0s
H    0     0                    1096926.0000 289279.892  73.6%     -    0s
H    0     0                    812965.00000 289279.892  64.4%     -    0s
H    0     0                

C:\Users\irako\AppData\Local\Temp\ipykernel_18780\978396970.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{



Root relaxation: objective 4.080493e+05, 467 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 408049.341    0  138          - 408049.341      -     -    0s
H    0     0                    9986615.0000 487162.807  95.1%     -    0s
     0     0 487162.807    0  166 9986615.00 487162.807  95.1%     -    0s
H    0     0                    5155811.0000 488215.147  90.5%     -    0s
H    0     0                    4817155.9999 488215.147  89.9%     -    0s
     0     0 488215.147    0  168 4817156.00 488215.147  89.9%     -    0s
     0     0 506185.479    0  183 4817156.00 506185.479  89.5%     -    0s
     0     0 507017.839    0  183 4817156.00 507017.839  89.5%     -    0s
H    0     0                    1512352.0000 507281.461  66.5%     -    0s
     0     2 507281.461    0  183 1512352.00 507281.461  66.5%     -    0s
H   56   

C:\Users\irako\AppData\Local\Temp\ipykernel_18780\978396970.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{


In [2]:
import matplotlib.pyplot as plt

# Generate and save histograms for each sheet
for sheet_name in results_df["Sheet"].unique():
    sheet_data = results_df[results_df["Sheet"] == sheet_name]

    plt.figure(figsize=(8, 5))
    plt.bar(sheet_data["Period"], sheet_data["y"], color='blue', alpha=0.7)
    plt.xlabel("Period")
    plt.ylabel("Inventory Level (S)")
    plt.title(f"Inventory Levels for {sheet_name}")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Save histogram as an image
    hist_filename = f"histogram_{sheet_name.replace(' ', '_')}.png"
    plt.savefig(hist_filename)
    print(f"Histogram saved: {hist_filename}")
    plt.close()

Histogram saved: histogram_6-periods_(1).png
Histogram saved: histogram_6-periods_(2).png
Histogram saved: histogram_12-periods_(1).png
Histogram saved: histogram_12-periods_(2).png
Histogram saved: histogram_24-periods_(1).png
Histogram saved: histogram_24-periods_(2).png
Histogram saved: histogram_52-periods_(1).png
Histogram saved: histogram_52-periods_(2).png
Histogram saved: histogram_104-periods_(1).png
Histogram saved: histogram_104-periods_(2).png
